In [7]:
import sys
import pygame
import numpy as np
import random
from time import sleep
# KEY:
# Human | Plays X | number representative : 1
# Comp  | Plays O | number representative : 5
# These numbers are chosen becuase 3x1 < 5, so if a row or column totals '3', it can only mean
#     that the row has 1 1 1 in it. and if the total is 7 then it is 5 1 1 in some order, and
#     so on. When we generalise this to N x N tic tac toe, we'll probably have to think of a
#     more elegant way to do this.

# The "ID"s for boxes are:
# 0 1 2
# 3 4 5
# 6 7 8

# Colors
white = (255,255,255)
black = (0,0,0)
red   = (255, 0, 0)


# Functions
#----------------------------

def announceResult(result):
    # Should we not pass the screen into this function? What is good programming practice?
    # Given that python is an interpreted language...
    clearScreen()
    if result == "comp":
        blitInCenter("You lost to a dumb machine", 20)
    elif result == "human":
        blitInCenter("Congratulations. You beat a dumb machine", 20)
    else:
        blitInCenter("You drew to a dumb machine", 20)
    pygame.display.update()
    sleep(2)
    
#----------------------------
def initEmptySpots():
    # Initialises the list of empty spots that we want to maintain. So that we can check if
    # a move is valid for a human, and also what moves the computer should consider when 
    # making its move.
    emptySpots = []
    for i in range(3):
        for j in range(3):
            emptySpots.append([i,j])
    return emptySpots

#----------------------------
def isWinner(player):
    # Checks if the "player" (either human or comp) has won the game:
    num = 1 if player=="human" else 5
    
    if 3*num in np.sum(board, axis = 0): # checking columns
        return True
    if 3*num in np.sum(board, axis = 1): # checking rows
        return True
    if np.trace(board) == 3*num: # checking leading diagonal
        return True  
    if board[2,0] + board[1,1] + board[0,2] == 3*num: # checking the not-leading diagonal
        return True
    return False

def isBoardFull():
    # should we not pass board in here as a function parameter?
    # What is good python programming ettiquete?!
    if 0 in board:
        return False
    else:
        return True
    
def isBoardEmpty():
    if np.sum(board) == 0:
        return True
    else:
        return False
    
def checkWinner():
    if isWinner("human"):
        return "human"
    elif isWinner("comp"):
        return "comp"
    elif isBoardFull():
        return "tie"
    return "play on"

#----------------------------
def getCoords(ID):
    # converts a number (0 to 8) into the row and column number of
    # the box that it represents.
    if ID > 8:
        raise Exception("Valid IDs are integers from 0 to 8 only")
    row = ID//3
    col = ID%3
    return [row, col]

#==============================================================================
def isSpaceEmpty(ID):
    return board[ID[0], ID[1]] == 0

#==============================================================================
def findBestMove(player):
    playerNum = 5 if player == "comp" else 1
    bestScore = -10
    if len(emptySpots) == 1:
        return emptySpots[0]
    for i in range(3):
        for j in range(3):
            if board[i,j] == 0:
                board[i,j] = 5
                score = minimax(board, "human")
                board[i,j] = 0
                if score > bestScore:
                    bestScore = score
                    bestID = [i,j]
    return bestID

#--------------------------------------------------------
# >> findBestMove
def minimax(board, player):
    # board: the game board in its current state
    # player: the player whose turn it is right now.
    
    # first check if there's a win, loss, or tie:
    result = checkWinner()
    if result == "comp": return 1 # comp wins when the score is high
    elif result == "human" : return -1 # human wins when score is low
    elif result == "tie" : return 0
    
    # if the game is not over, decide the best move:
    if player == "comp": # comp wants to maximise the score
        bestScore = -10
        for i in range(3):
            for j in range(3):
                if board[i,j] == 0:
                    board[i,j] = 5 # temporarily play comp move on that spot.
                    score = minimax(board, "human")
                    board[i,j] = 0 # remove the temp comp move you just played.
                    if score > bestScore:
                        bestScore = score
        return bestScore # so this is the best (highest) possible score a 
                         #comp can get from this state onwards
    
    elif player == "human": # human wants to minimize the score
        bestScore = 10
        for i in range(3):
            for j in range(3):
                if board[i,j] == 0:
                    board[i,j] = 1
                    score = minimax(board, "comp")
                    board[i,j] = 0
                    if score < bestScore:
                        bestScore = score
        return bestScore # lowest possible score a human can get
    
#==============================================================================
def play(currPlayer):
    if currPlayer == "human":
        playHuman() ##
    else:
        playComputer() ##
        
#-----------------------------
# >> play:
def playHuman():
    waitingForClick = True
    
    while waitingForClick:
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
                
            if event.type == pygame.MOUSEBUTTONDOWN:
                mousePos = pygame.mouse.get_pos()
                boxID = checkWhichBox(mousePos) ##
                if boxID == "no box":
                    continue
                if not isSpaceEmpty(boxID):
                    continue
                
                updateBoards(boxID, "human") ##
                waitingForClick = False
                
#-----------------------------
# >> play >> playHuman
def checkWhichBox(mousePos):
    for idx,box in enumerate(boxes):
        if box.collidepoint(mousePos):
            coords = getCoords(idx)
            break
    else:
        return "no box"
    return coords
#----------------------------------------------------------
# >> play
def playComputer():
    sleep(1)
    if isBoardEmpty():
        theMove = [1,1]
        # A little shortcut here, otherwise the computer takes 
        # a while to figure out that the initial best move is in the center
    else:
        theMove = findBestMove("comp")
    updateBoards(theMove, "comp")
    
#==============================================================================
# >> play >> playHuman, playcomputer
def updateBoards(boxID, player):
    markBoard(boxID, player) # updating the numpy array
    emptySpots.remove(boxID) # updating the list of empty spots
    
    boxNum = boxID[0]*3 + boxID[1]
    #print(player, boxNum)
    if player == "human"  : drawXinBox(boxes[boxNum])
    elif player == "comp" : drawOinBox(boxes[boxNum])
        
#--------------------------------------------------------
# >> updateBoards
def markBoard(boxID, player):
    playerNumber = 1 if player == "human" else 5
    board[boxID[0], boxID[1]] = playerNumber
    
#--------------------------------------------------------
# >> updateBoards
def drawXinBox(box):
    centerX = box.center[0]
    centerY = box.center[1]
    pygame.draw.line(window, red, (centerX-40, centerY-40), (centerX+40, centerY+40), 4 )
    pygame.draw.line(window, red, (centerX+40, centerY-40), (centerX-40, centerY+40), 4 )
    
def drawOinBox(box):
    pygame.draw.circle(window, black, box.center, 40, 3)
    

#==============================================================================
def changePlayer(x):
    newPlayer = "comp" if x == "human" else "human"
    return newPlayer

#-----------------------------
def makeGrid():
    boxes = []
    for i in range(9):
        row = i//3
        col = i%3
        box = pygame.draw.rect(window, 
                               (255,255,255), 
                               (20 + col*170, 20 + row*170 ,150,150))
        boxes.append(box)
    pygame.display.update()
    return boxes

#-----------------------------
def flipCoinAnimation():
    # fill this stuff in later
    pass
    
def chooseStartingPlayer():
    flipCoinAnimation()
    n = random.random() # n is a random float between 0 and 1
    
    currPlayer  = "human" if n<0.5 else "comp"
    displayText = "Your Turn First" if n<0.5 else "Computer Goes First"
    
    blitInCenter(displayText, 20)
    pygame.display.update()
    sleep(2)
    clearScreen()
    return currPlayer

#-----------------------------
def blitInCenter(text, size, color = white):
    wordFont = pygame.font.Font("freesansbold.ttf", size)
    message  = wordFont.render(text, True, color, black)
    window.blit(message, message.get_rect(center = (530/2, 530/2)))
    
def clearScreen():
    window.fill(black)
    
def waitForClick():
    clicked = False
    while not clicked:
        for event in pygame.event.get():
            if event.type == pygame.MOUSEBUTTONDOWN:
                clicked = True
    
def clickToBegin():
    blitInCenter("...CLICK ANYWHERE TO CONTINUE...", 15)
    pygame.display.update()
    waitForClick()
    clearScreen()
    

In [8]:
if __name__ == "__main__":
    pygame.init()
    window = pygame.display.set_mode((530,530))
    pygame.display.set_caption('Tic Tac Toe')

    clickToBegin()
    currPlayer = chooseStartingPlayer()

    boxes = makeGrid()
    board = np.zeros((3,3), dtype = int)
    emptySpots = initEmptySpots()

    # Begin
    run = True
    while run:
        # check for a winner
        result = checkWinner()
        if result != "play on":
            sleep(1)
            break
        play(currPlayer)
        currPlayer = changePlayer(currPlayer)
        #print(board)
        pygame.display.update()


    announceResult(result)
    pygame.quit()
    